# Twirling of effective noise channel

In [1]:
from pyquil.quil import *
from pyquil.api import get_qc
from pyquil.gates import *
from pyquil.latex import display, to_latex
from pyquil.simulation.tools import lifted_gate, program_unitary, lifted_gate_matrix

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from functions import *

## Generation of Haar random circuits

In [4]:
target_qubits = [0,1]
prog = give_random_two_qubit_circuit(target_qubits)
prog.instructions

[<Gate RZ(0.3191438948870753) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(2.9492502462271526) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(1.759006675293393) 0>,
 <Gate RZ(1.8633361847266914) 1>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(0.5129130863003744) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate RZ(4.818593091141783) 1>,
 <Gate RZ(pi) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(pi/2) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate CZ 1 0>,
 <Gate RZ(pi) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(pi/2) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(0.8614556850672044) 0>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(4.23772499655575) 1>,
 <Gate RX(pi/2) 1>,
 <Gate CZ 0 1>,
 <Gate RZ(pi) 1>,
 <Gate RX(pi/2) 1>,
 <Gate RZ(3.6297060818408315) 1>,
 <Gate RX(-pi/2) 1>,
 <Gate RZ(pi) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(pi/2) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate CZ 1 0>,
 <Gate RZ(pi) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(pi/2) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(6.015214063260408) 0>,
 <Gate RX(pi/2) 0>,
 <Gate RZ(0.7693696294505615) 0>,
 <Gate RX(-pi/2) 0>,
 <Gate RZ(4.586757392302429) 0>,
 <Gate RZ(5.

### Find the (4,4)-matrix representation of each gate

In [5]:
matrix_list = []
for gate in prog.instructions:
    matrix_list.append( program_unitary( Program(gate), n_qubits = 2) )
matrix_list

[array([[0.98729539-0.15889561j, 0.        +0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.98729539+0.15889561j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.98729539-0.15889561j, 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.        +0.j        , 0.98729539+0.15889561j]]),
 array([[0.70710678+0.j        , 0.        -0.70710678j,
         0.        +0.j        , 0.        +0.j        ],
        [0.        -0.70710678j, 0.70710678+0.j        ,
         0.        +0.j        , 0.        +0.j        ],
        [0.        +0.j        , 0.        +0.j        ,
         0.70710678+0.j        , 0.        -0.70710678j],
        [0.        +0.j        , 0.        +0.j        ,
         0.        -0.70710678j, 0.70710678+0.j        ]]),
 array([[0.09602303-0.99537911j, 0.        +0.j        ,
         0.        